# Reading netCDF data
- requires [numpy](http://numpy.scipy.org) and netCDF/HDF5 C libraries.
- Github site: https://github.com/Unidata/netcdf4-python
- Online docs: http://unidata.github.io/netcdf4-python/
- Based on Konrad Hinsen's old [Scientific.IO.NetCDF](http://dirac.cnrs-orleans.fr/plone/software/scientificpython/) API, with lots of added netcdf version 4 features.
- Developed by Jeff Whitaker at NOAA, with many contributions from users.

## Interactively exploring a netCDF File

Let's explore a netCDF file from the *Atlantic Real-Time Ocean Forecast System*

first, import netcdf4-python and numpy

In [1]:
pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00


In [2]:
import netCDF4
import numpy as np

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ls /content/drive/Shareddrives/TAMIDS\ Data\ Science\ Competition\ 2024/0_data

 Budget_TimeSeries.mat                                                    LRD_Noise_USEastCoast.mat
 c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D_1709513365903.nc   Master_D23_Github.m
'CSVs from .mat files'/                                                   Plot_RatesallStations.m
 data_copernicus_adt_1993_2023.nc


## Create a netCDF4.Dataset object
- **`f`** is a `Dataset` object, representing an open netCDF file.
- printing the object gives you summary information, similar to *`ncdump -h`*.

In [5]:
dataset = '/content/drive/Shareddrives/TAMIDS Data Science Competition 2024/0_data/c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D_1709513365903.nc'

In [6]:
# f = netCDF4.Dataset('data/rtofs_glo_3dz_f006_6hrly_reg3.nc')
f = netCDF4.Dataset(dataset)
print(f)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.11
    title: DT merged two satellites Global Ocean Gridded SSALTO/DUACS Sea Surface Height L4 product and derived variables
    institution: CLS, CNES
    source: Altimetry measurements
    history: 2021-07-09 08:53:58Z: Creation
    contact: http://climate.copernicus.eu/c3s-user-service-desk
    references: http://climate.copernicus.eu
    comment: Sea Surface Height measured by Altimetry and derived variables
    subset:source: ARCO data downloaded from the Marine Data Store using the MyOcean Data Portal
    subset:productId: SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057
    subset:datasetId: c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D_202112
    subset:date: 2024-03-04T00:49:25.913Z
    dimensions(sizes): latitude(720), longitude(1440), time(1)
    variables(dimensions): float32 adt(time, latitude, longitude), float32 err_sla(time, latitude, longitude), float32 err_ugosa(t

In [7]:
# adt: absolute dynamic topography
# sla: Sea level anomaly; sea_surface_height_above_sea_level.
# err_sla: sea_surface_height_above_sea_level standard_error
# ugosa: Geostrophic velocity anomalies: zonal component
# vgosa: Geostrophic velocity anomalies: meridian component
# flag_ice [0,1]: Ice Flag for a 15% criterion of ice concentration - status_flag
# tpa_correction: TOPEX-A instrumental drift correction derived from altimetry and tide gauges global comparisons (WCRP Sea Level Budget Group, 2018); sea_surface_height_above_sea_level
#


In [8]:
import xarray as xr

# Load the dataset
ds = xr.open_dataset(dataset)

# Calculate the average ADT across all longitudes and latitudes for the given time step
adt_avg = ds['adt'].mean(dim=['latitude', 'longitude'])

# Output the average ADT value
print(adt_avg.values)



[0.38769194]


## Access a netCDF variable
- variable objects stored by name in **`variables`** dict.
- print the variable yields summary info (including all the attributes).
- no actual data read yet (just have a reference to the variable object with metadata).

In [9]:
print(f.variables.keys()) # get all variable names
temp = f.variables['adt']  # temperature variable
print(temp)

dict_keys(['adt', 'err_sla', 'err_ugosa', 'err_vgosa', 'flag_ice', 'latitude', 'longitude', 'sla', 'time', 'tpa_correction', 'ugos', 'ugosa', 'vgos', 'vgosa'])
<class 'netCDF4._netCDF4.Variable'>
float32 adt(time, latitude, longitude)
    units: m
    _FillValue: nan
    standard_name: sea_surface_height_above_geoid
    long_name: Absolute dynamic topography
unlimited dimensions: 
current shape = (1, 720, 1440)
filling off


In [10]:
time = f.variables['time']  # temperature variable
print(time)

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: Time
    units: seconds since 1970-01-01 00:00:00
    calendar: gregorian
    axis: T
unlimited dimensions: 
current shape = (1,)
filling off


## List the Dimensions

- All variables in a netCDF file have an associated shape, specified by a list of dimensions.
- Let's list all the dimensions in this netCDF file.
- Note that the **`MT`** dimension is special (*`unlimited`*), which means it can be appended to.

In [11]:
for d in f.dimensions.items():
    print(d)

('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 720)
('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 1440)
('time', <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 1)


Each variable has a **`dimensions`** and a **`shape`** attribute.

In [12]:
temp.dimensions

('time', 'latitude', 'longitude')

In [13]:
temp.shape

(1, 720, 1440)

### Each dimension typically has a variable associated with it (called a *coordinate* variable).
- *Coordinate variables* are 1D variables that have the same name as dimensions.
- Coordinate variables and *auxiliary coordinate variables* (named by the *coordinates* attribute) locate values in time and space.

In [14]:
time = f.variables['time']
latitude = f.variables['latitude']
longitude = f.variables['longitude']
print(time)
print(latitude)
print(longitude)

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: Time
    units: seconds since 1970-01-01 00:00:00
    calendar: gregorian
    axis: T
unlimited dimensions: 
current shape = (1,)
filling off
<class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    standard_name: latitude
    long_name: Latitude
    units: degrees_north
    unit_long: Degrees North
    axis: Y
    valid_min: -89.875
    valid_max: 89.875
unlimited dimensions: 
current shape = (720,)
filling off
<class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    standard_name: longitude
    long_name: Longitude
    units: degrees_east
    unit_long: Degrees East
    axis: X
unlimited dimensions: 
current shape = (1440,)
filling off


## Accessing data from a netCDF variable object

- netCDF variables objects behave much like numpy arrays.
- slicing a netCDF variable object returns a numpy array with the data.
- Boolean array and integer sequence indexing behaves differently for netCDF variables than for numpy arrays. Only 1-d boolean arrays and integer sequences are allowed, and these indices work independently along each dimension (similar to the way vector subscripts work in fortran).

In [15]:
time = time[:]  # Reads the netCDF variable MT, array of one element
print(time)

[1.686096e+09]


In [16]:
latitude = latitude[:] # examine depth array
print(latitude)

[-89.875 -89.625 -89.375 -89.125 -88.875 -88.625 -88.375 -88.125 -87.875
 -87.625 -87.375 -87.125 -86.875 -86.625 -86.375 -86.125 -85.875 -85.625
 -85.375 -85.125 -84.875 -84.625 -84.375 -84.125 -83.875 -83.625 -83.375
 -83.125 -82.875 -82.625 -82.375 -82.125 -81.875 -81.625 -81.375 -81.125
 -80.875 -80.625 -80.375 -80.125 -79.875 -79.625 -79.375 -79.125 -78.875
 -78.625 -78.375 -78.125 -77.875 -77.625 -77.375 -77.125 -76.875 -76.625
 -76.375 -76.125 -75.875 -75.625 -75.375 -75.125 -74.875 -74.625 -74.375
 -74.125 -73.875 -73.625 -73.375 -73.125 -72.875 -72.625 -72.375 -72.125
 -71.875 -71.625 -71.375 -71.125 -70.875 -70.625 -70.375 -70.125 -69.875
 -69.625 -69.375 -69.125 -68.875 -68.625 -68.375 -68.125 -67.875 -67.625
 -67.375 -67.125 -66.875 -66.625 -66.375 -66.125 -65.875 -65.625 -65.375
 -65.125 -64.875 -64.625 -64.375 -64.125 -63.875 -63.625 -63.375 -63.125
 -62.875 -62.625 -62.375 -62.125 -61.875 -61.625 -61.375 -61.125 -60.875
 -60.625 -60.375 -60.125 -59.875 -59.625 -59.375 -5

In [17]:
longitude = longitude[:] # examine depth array
print(longitude)

[-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]


## What is the sea surface temperature and salinity at 50N, 140W?
### Finding the latitude and longitude indices of 50N, 140W

- The `X` and `Y` dimensions don't look like longitudes and latitudes
- Use the auxilary coordinate variables named in the `coordinates` variable attribute, `Latitude` and `Longitude`

In [18]:
lat, lon = f.variables['latitude'], f.variables['longitude']
print(lat)

<class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    standard_name: latitude
    long_name: Latitude
    units: degrees_north
    unit_long: Degrees North
    axis: Y
    valid_min: -89.875
    valid_max: 89.875
unlimited dimensions: 
current shape = (720,)
filling off


Aha!  So we need to find array indices `iy` and `ix` such that `Latitude[iy, ix]` is close to 50.0 and `Longitude[iy, ix]` is close to -140.0 ...

In [19]:
# extract lat/lon values (in degrees) to numpy arrays
latvals = lat[:]; lonvals = lon[:]
# a function to find the index of the point closest pt
# (in squared distance) to give lat/lon value.
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2
    # 1D index of minimum dist_sq element
    minindex_flattened = dist_sq.argmin()
    # Get 2D index for latvals and lonvals arrays from 1D index
    return np.unravel_index(minindex_flattened, lats.shape)
iy_min, ix_min = getclosest_ij(latvals, lonvals, 50., -140)

ValueError: operands could not be broadcast together with shapes (720,) (1440,) 

### Now we have all the information we need to find our answer.


```
|----------+--------|
| Variable |  Index |
|----------+--------|
| MT       |      0 |
| Depth    |      0 |
| Y        | iy_min |
| X        | ix_min |
|----------+--------|
```

### What is the sea surface temperature and salinity at the specified point?

In [ ]:
sal = f.variables['salinity']
# Read values out of the netCDF file for temperature and salinity
print('%7.4f %s' % (temp[0,0,iy_min,ix_min], temp.units))
print('%7.4f %s' % (sal[0,0,iy_min,ix_min], sal.units))

## Remote data access via openDAP

- Remote data can be accessed seamlessly with the netcdf4-python API
- Access happens via the DAP protocol and DAP servers, such as TDS.
- many formats supported, like GRIB, are supported "under the hood".

The following example showcases some nice netCDF features:

1. We are seamlessly accessing **remote** data, from a TDS server.
2. We are seamlessly accessing **GRIB2** data, as if it were netCDF data.
3. We are generating **metadata** on-the-fly.

In [ ]:
import datetime
date = datetime.datetime.now()
# build URL for latest synoptic analysis time
URL = 'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg/GFS_Global_0p5deg_%04i%02i%02i_%02i%02i.grib2/GC' %\
(date.year,date.month,date.day,6*(date.hour//6),0)
# keep moving back 6 hours until a valid URL found
validURL = False; ncount = 0
while (not validURL and ncount < 10):
    print(URL)
    try:
        gfs = netCDF4.Dataset(URL)
        validURL = True
    except RuntimeError:
        date -= datetime.timedelta(hours=6)
        ncount += 1

In [ ]:
# Look at metadata for a specific variable
# gfs.variables.keys() will show all available variables.
sfctmp = gfs.variables['Temperature_surface']
# get info about sfctmp
print(sfctmp)
# print coord vars associated with this variable
for dname in sfctmp.dimensions:
    print(gfs.variables[dname])

##Missing values
- when `data == var.missing_value` somewhere, a masked array is returned.
- illustrate with soil moisture data (only defined over land)
- white areas on plot are masked values over water.

In [ ]:
soilmvar = gfs.variables['Volumetric_Soil_Moisture_Content_depth_below_surface_layer']
# flip the data in latitude so North Hemisphere is up on the plot
soilm = soilmvar[0,0,::-1,:]
print('shape=%s, type=%s, missing_value=%s' % \
      (soilm.shape, type(soilm), soilmvar.missing_value))
import matplotlib.pyplot as plt
%matplotlib inline
cs = plt.contourf(soilm)

##Packed integer data
There is a similar feature for variables with `scale_factor` and `add_offset` attributes.

- short integer data will automatically be returned as float data, with the scale and offset applied.  

## Dealing with dates and times
- time variables usually measure relative to a fixed date using a certain calendar, with units specified like ***`hours since YY:MM:DD hh-mm-ss`***.
- **`num2date`** and **`date2num`** convenience functions provided to convert between these numeric time coordinates and handy python datetime instances.  
- **`date2index`** finds the time index corresponding to a datetime instance.

In [ ]:
from netCDF4 import num2date, date2num, date2index
timedim = sfctmp.dimensions[0] # time dim name
print('name of time dimension = %s' % timedim)
times = gfs.variables[timedim] # time coord var
print('units = %s, values = %s' % (times.units, times[:]))

In [ ]:
dates = num2date(times[:], times.units)
print([date.strftime('%Y-%m-%d %H:%M:%S') for date in dates[:10]]) # print only first ten...

###Get index associated with a specified date, extract forecast data for that date.

In [ ]:
from datetime import datetime, timedelta
date = datetime.now() + timedelta(days=3)
print(date)
ntime = date2index(date,times,select='nearest')
print('index = %s, date = %s' % (ntime, dates[ntime]))

###Get temp forecast for Boulder (near 40N, -105W)
- use function **`getcloses_ij`** we created before...

In [ ]:
lats, lons = gfs.variables['lat'][:], gfs.variables['lon'][:]
# lats, lons are 1-d. Make them 2-d using numpy.meshgrid.
lons, lats = np.meshgrid(lons,lats)
j, i = getclosest_ij(lats,lons,40,-105)
fcst_temp = sfctmp[ntime,j,i]
print('Boulder forecast valid at %s UTC = %5.1f %s' % \
      (dates[ntime],fcst_temp,sfctmp.units))

##Simple multi-file aggregation

What if you have a bunch of netcdf files, each with data for a different year, and you want to access all the data as if it were in one file?

In [ ]:
!ls -ldgG data/prmsl*nc

**`MFDataset`** uses file globbing to patch together all the files into one big Dataset.
You can also pass it a list of specific files.

Limitations:

- It can only  aggregate the data along the leftmost dimension of each variable.
- only works with `NETCDF3`, or `NETCDF4_CLASSIC` formatted files.
- kind of slow.

In [ ]:
mf = netCDF4.MFDataset('data/prmsl*nc')
times = mf.variables['time']
dates = num2date(times[:],times.units)
print('starting date = %s' % dates[0])
print('ending date = %s'% dates[-1])
prmsl = mf.variables['prmsl']
print('times shape = %s' % times.shape)
print('prmsl dimensions = %s, prmsl shape = %s' %\
     (prmsl.dimensions, prmsl.shape))

## Closing your netCDF file

It's good to close netCDF files, but not actually necessary when Dataset is open for read access only.


In [ ]:
f.close()
gfs.close()

##That's it!

Now you're ready to start exploring your data interactively.

To be continued with **Writing netCDF data** ....